In [ ]:
# data cleaning -> graph construction -> queries -> output for sql processing? -> data postprocessing? -> visualization?

In [1]:
!pip3 install py2neo
!pip3 install pandas
!pip3 install requests
!pip3 install bs4
!pip3 install pymongo
# !pip3 install urllib
!pip install rake-nltk
!pip install pygooglenews

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=db86a510c083d0c3fbd31ec9cc6b026651d9186360ef466962cf1d91f3c0ed18
  Stored in directory: c:\users\mansi\appdata\local\pip\cache\wheels\19\f5\6d\a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=45190 sha256=6303399f35596c669e18c33ad424d030ad918bcbd562cbf02d8c8fd4730c2c3b
  Stored in directory: c:\users\mansi\appdata\local\pip\cache\wheels\60\4c\d6\d096a99d3d96af9af149c7d82c5969a0896bcce1ac2a15ccd6
Successfully built feedparser


In [2]:
!pip3 install nltk

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mansi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mansi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# all libraries and static data
from py2neo import Graph
from rake_nltk import Rake
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import json
from pymongo import MongoClient
import urllib
from pygooglenews import GoogleNews
from google.colab import files #added for colab
import io #added for colab

g = Graph("neo4j://awesome-hw.sdsc.edu", auth=("neo4j", "b!Q{@q9m"))
r = Rake()
# model = KeyBERT(model="distilbert-base-nli-mean-tokens")
gn = GoogleNews(lang = 'en', country = 'US')

In [ ]:
# query = "MATCH (n:technology) RETURN n LIMIT 25"

In [ ]:
# g.run(query).to_table()

In [ ]:
# patent_result = []
# for i, row in patentdb.iterrows():
#     if not pd.isna(row['abstract']):  #check if the abstract text exists
#         r.extract_keywords_from_text(row['abstract'])
#         keywords = set(r.get_ranked_phrases()) 
#         patent_result.append([row['patentid'], "{" + ", ".join(list(keywords)) + "}"])
        
# patent_keywords_df = pd.DataFrame(patent_result)
# patent_keywords_df.to_csv('patentdb_keywords.csv', header=['patentid', 'keywords'], index=False)

In [ ]:
uploaded = files.upload() #for google colab

Saving Result_12.csv to Result_12.csv


In [ ]:
# patentdb = pd.read_csv('patentdb.csv')
sbirdb = pd.read_csv(io.BytesIO(uploaded['Result_12.csv']), usecols = ['recordid','company','award_title','award_year','award_amount','abstract'])

In [ ]:
sbirdf = pd.DataFrame(sbirdb)

In [ ]:
sbirdf.head()

,recordid,company,award_title,award_year,award_amount,abstract
0,166611,Physical Optics Corporation,A Novel Technique for 4-Dimensional Atmospheri...,1994,59981.0,Conventional imaging systems do not have any d...
1,166612,Physical Optics Corporation,A NOVEL 3-D DISPLAY CONCEPT AND ITS APPLIATION...,1994,282121.0,HEAD MOUNTED DISPLAY SYSTEMS (HMDS) ARE RECOGN...
2,166613,Physical Optics Corporation,A High-Speed and Highly Sensitive Wide-Area Su...,1994,375000.0,NaN
3,166614,Physical Optics Corporation,Fresnel Image Wavefront Sensor With High Sensi...,1994,375000.0,To reduce the complexity of Hartmann sensor op...
4,166615,Physical Optics Corporation,HIGHLY SENSITIVE FIBER OPTIC SENSOR FOR CANCER...,1994,385000.0,FLUORESENCE-BASED PHOTODYNAMIC THERAPY HAS PRO...


In [ ]:
def keyword_extractor(input_data):
  if not pd.isna(input_data):
    if isinstance(input_data,str):
      r.extract_keywords_from_text(input_data)
      temp = set(r.get_ranked_phrases())
      return(list(temp)[:3])

In [ ]:
sbirdf['keyword_list'] = sbirdf['abstract'].apply(keyword_extractor)

In [ ]:
sbirdf = sbirdf[sbirdf['abstract'].notna()]
sbirdf.head()

,recordid,company,award_title,award_year,award_amount,abstract,keyword_list
0,166611,Physical Optics Corporation,A Novel Technique for 4-Dimensional Atmospheri...,1994,59981.0,Conventional imaging systems do not have any d...,"[performance, system, matched filter]"
1,166612,Physical Optics Corporation,A NOVEL 3-D DISPLAY CONCEPT AND ITS APPLIATION...,1994,282121.0,HEAD MOUNTED DISPLAY SYSTEMS (HMDS) ARE RECOGN...,"[inherent weakneses, layers, producing]"
3,166614,Physical Optics Corporation,Fresnel Image Wavefront Sensor With High Sensi...,1994,375000.0,To reduce the complexity of Hartmann sensor op...,"[long, wavefront sensor, method]"
4,166615,Physical Optics Corporation,HIGHLY SENSITIVE FIBER OPTIC SENSOR FOR CANCER...,1994,385000.0,FLUORESENCE-BASED PHOTODYNAMIC THERAPY HAS PRO...,"[develop, need, properly process]"
5,166616,Physical Optics Corporation,Radiation Hard SCIntIIIatIng Detector Based on...,1994,61197.0,PHYSCIAL OPTICS CORPORATION (POC) PROPOSES TO ...,"[rugged, scintillation efficiency, new generat..."


In [ ]:
sbirdf.to_csv('sbir_with_keywords.csv', index=False)

In [ ]:
files.download("sbir_with_keywords.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,recordid,company,award_title,abstract,keyword_list
0,166611,Physical Optics Corporation,A Novel Technique for 4-Dimensional Atmospheri...,Conventional imaging systems do not have any d...,"[strong background, image focal plane, discrim..."
1,166612,Physical Optics Corporation,A NOVEL 3-D DISPLAY CONCEPT AND ITS APPLIATION...,HEAD MOUNTED DISPLAY SYSTEMS (HMDS) ARE RECOGN...,"[good device, maintain, distance]"
3,166614,Physical Optics Corporation,Fresnel Image Wavefront Sensor With High Sensi...,To reduce the complexity of Hartmann sensor op...,"[method, various advantages, causes displacement]"
4,166615,Physical Optics Corporation,HIGHLY SENSITIVE FIBER OPTIC SENSOR FOR CANCER...,FLUORESENCE-BASED PHOTODYNAMIC THERAPY HAS PRO...,"[data, method allows detection, increase signal]"
5,166616,Physical Optics Corporation,Radiation Hard SCIntIIIatIng Detector Based on...,PHYSCIAL OPTICS CORPORATION (POC) PROPOSES TO ...,"[combination, diamond, ray sources]"


In [ ]:
def keywords_to_be_extracted(keyword_list):
    recordID_articles = []
    keywordlist_articles_compiled = {}
    for keyword in keyword_list:
        recordID_articles.append(extract_news(keyword))
    keywordlist_articles_compiled['keywordlist_news'] = recordID_articles
    return keywordlist_articles_compiled
            
def extract_news(keyword):
  search = gn.search(keyword)
  keyword_articles_compiled = {}
  keyword_articles = []
  if len(search['entries'])>5:
    for i in range(1,4):
      article = {}
      article['article_title'] = search['entries'][i]['title']
      article['article_by'] = search['entries'][i]['source']['title']
      article['article_link'] = search['entries'][i].link
      article['published_on'] = search['entries'][i].published
      keyword_articles.append(article)
  keyword_articles_compiled[keyword] = keyword_articles
  return keyword_articles_compiled

In [ ]:
def news_webscraper(keywords):
  if len(keywords)>0:
    news = keywords_to_be_extracted(keywords)
    return news


In [ ]:
sbirdf['news'] = sbirdf['keyword_list'].apply(news_webscraper)

In [ ]:
# info_dict = {}
# rows,cols = sbir_keywords_df.shape
# for r in range(rows):
#     row = sbir_keywords_df.loc[r]
#     keyword_list = list(row['keywords'])
#     recordID = row['recordid']
#     if len(keyword_list)>0:
#       info_dict[recordID] = keywords_to_be_extracted(keyword_list)


In [ ]:
# scraped = pd.DataFrame(info_dict.items(),columns = ['recordid','scraped'])  
# print(scraped[scraped['recordid'] == 166611].scraped)
sbirdf.head()

,recordid,company,award_title,abstract,keyword_list,news
0,166611,Physical Optics Corporation,A Novel Technique for 4-Dimensional Atmospheri...,Conventional imaging systems do not have any d...,"[strong background, image focal plane, discrim...",None
1,166612,Physical Optics Corporation,A NOVEL 3-D DISPLAY CONCEPT AND ITS APPLIATION...,HEAD MOUNTED DISPLAY SYSTEMS (HMDS) ARE RECOGN...,"[good device, maintain, distance]",None
3,166614,Physical Optics Corporation,Fresnel Image Wavefront Sensor With High Sensi...,To reduce the complexity of Hartmann sensor op...,"[method, various advantages, causes displacement]",None
4,166615,Physical Optics Corporation,HIGHLY SENSITIVE FIBER OPTIC SENSOR FOR CANCER...,FLUORESENCE-BASED PHOTODYNAMIC THERAPY HAS PRO...,"[data, method allows detection, increase signal]",None
5,166616,Physical Optics Corporation,Radiation Hard SCIntIIIatIng Detector Based on...,PHYSCIAL OPTICS CORPORATION (POC) PROPOSES TO ...,"[combination, diamond, ray sources]",None


In [ ]:
ex.to_csv('scraped.csv', index=False)
files.download("scraped.csv") #for google colab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[]


In [ ]:
# mongoDB connection to jupyter 

In [ ]:
def connect_mongodb(mongo_url,port):
    try:
        client = MongoClient(mongo_url,port)
        print("Connected to MongoDB Atlas")
        return client
    except:
        print("Couldn't connect")
        
def insert_data(table,data):
    try:
        table.insert_many(data)
        print("Records inserted in MongoDB")
    except:
        print("Could not insert")

def read_json_file(file):
    data = []
    for f in file:
        data.append(json.loads(f))
    return data

In [ ]:
client = connect_mongodb('localhost',27017)
db = client.demo
collection = db.info
# file = open('samplenew1.json','r') #do we need this? already creating a file in convert_to_json
# data = read_json_file(file)
# insert_data(collection,data)

Connected to MongoDB Atlas


In [ ]:
cursor = collection.find({'cuisine':{'$eq':'Bakery'}},{'_id':0})
cursor.count()
cursor.next()

{'title': 'PHOTOS: I visited the American Girl Doll cafe. - Business Insider', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': 'PHOTOS: I visited the American Girl Doll cafe. - Business Insider'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.google.com/__i/rss/rd/articles/CBMiYWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vcGhvdG9zLWktdmlzaXRlZC10aGUtYW1lcmljYW4tZ2lybC1kb2xsLXBsYWNlLWNhZmUtcmV0YWlsLTIwMjItMTHSAWVodHRwczovL3d3dy5idXNpbmVzc2luc2lkZXIuY29tL3Bob3Rvcy1pLXZpc2l0ZWQtdGhlLWFtZXJpY2FuLWdpcmwtZG9sbC1wbGFjZS1jYWZlLXJldGFpbC0yMDIyLTExP2FtcA?oc=5'}], 'link': 'https://news.google.com/__i/rss/rd/articles/CBMiYWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci5jb20vcGhvdG9zLWktdmlzaXRlZC10aGUtYW1lcmljYW4tZ2lybC1kb2xsLXBsYWNlLWNhZmUtcmV0YWlsLTIwMjItMTHSAWVodHRwczovL3d3dy5idXNpbmVzc2luc2lkZXIuY29tL3Bob3Rvcy1pLXZpc2l0ZWQtdGhlLWFtZXJpY2FuLWdpcmwtZG9sbC1wbGFjZS1jYWZlLXJldGFpbC0yMDIyLTExP2FtcA?oc=5', 'id': 'CBMiYWh0dHBzOi8vd3d3LmJ1c2luZXNzaW5zaWRlci